<img src="imgs/logo-spegc.svg" width=30%>

# Redes Neuronales

Una red neuronal es un clasificador. Su funcionamiento se basa en la combinación de hiperplanos que acotan regiones de activación del espacio de muestras.

<img src="images/red.svg" width=30%>

### Mini-batch

En cada ciclo de actualización de los pesos de la red no es necesario calcular el gradiente de descenso para cada muestra. Basta con escoger un lote variado de muestras (mini-batch) y calcular el gradiente resultante de éste. 

### Loss

La pérdida o *loss* es el valor o error resultante de comparar las predicciones de la red con las etiquetas de las muestras.

### One-hot

Cuando se clasifica en más de dos categorías es necesario que la red tenga tantas neuronas en la capa de salida como clases haya. Por tanto, la activación de cada una de estas neuronas de salida corresponderá con una clase. Por ejemplo, si la red tiene cuatro salidas y se activa la tercera, el vector resultante podría ser algo como: [0.002,0.008,0.95,0.04]. Como normalmente las etiquetas de las muestras están en un formato numérico entero (0,1,2,3...), no podremos realizar la comparación de las salidas con las etiquetas. Así que transformamos la etiqueta en un vector de todo ceros excepto el lugar correspondiente a la etiqueta, el cual tendrá un valor de 1. Por ejemplo: $2\rightarrow [0,0,1,0]$. (Recordemos que las etiquetas comienzan en 0). Esta función de transformación se denomina **One-hot**.

### Época

Cada vez que entrenamos una red con todos los lotes o *mini-batches* de nuestro dataset de entrenamiento decimos que hemos completado una **época**. Durante el proceso de entrenamiento de una red se suelen completar múltiples épocas.

### Softmax

La función softmax es parecida a la sigmoide, ésta también convierte un valor entre menos infinito y más infinito en un valor entre cero y uno. La diferencia está en que la función softmax no trabaja sobre un valor sino sobre un vector. De esta forma, convierte todas las componentes de un vector en valores entre cero y uno, pero, además, garantiza que la suma de todos estos valores sea uno. La función softmax se emplea en la capa de salida de una red, y convierte de esta forma sus salidas en una distribución de probabilidades.

$$ Softmax(\textbf{v})_i = \frac{e^{v_i}}{\sum_{j=1}^{n} e^{v_j}} $$


## Ejemplo

In [59]:
import numpy as np
from NN4Teaching import nn4t

data = np.genfromtxt('data/iris.data', delimiter=",")
np.random.shuffle(data)
x_data = data[:, 0:4].astype('f4')
y_data = nn4t.one_hot(data[:, 4].astype(int), 3)

net = nn4t.Net(layers=[4, 5, 3])

for i in range(900):
    net.train(x_data, y_data)
    
    if i%50 == 0:
        print("Epoch: ", i)
        print(net.loss(x_data, y_data))

for x, y in zip(x_data[:15], y_data[:15]):
    print(y)
    print(net.output(x))

Epoch:  0
nan
Epoch:  50
nan
Epoch:  100
nan
Epoch:  150
nan
Epoch:  200
nan
Epoch:  250
nan
Epoch:  300
nan
Epoch:  350
nan
Epoch:  400
nan
Epoch:  450
nan
Epoch:  500
nan
Epoch:  550
nan
Epoch:  600
nan
Epoch:  650
nan
Epoch:  700
nan
Epoch:  750
nan
Epoch:  800
nan
Epoch:  850
nan
[1. 0. 0.]
[nan nan nan]
[1. 0. 0.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[0. 0. 1.]
[nan nan nan]
[1. 0. 0.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[0. 0. 1.]
[nan nan nan]
[0. 0. 1.]
[nan nan nan]
[1. 0. 0.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[0. 0. 1.]
[nan nan nan]
[0. 1. 0.]
[nan nan nan]
[1. 0. 0.]
[nan nan nan]


### Ejercicios

- Entrena la red mediante mini-batchs.
- Crea un gráfico del loss.
- Finaliza el entrenamiento cuando la curva del loss se estabilice.
- Divide el dataset en entrenamiento y test.
- Calcula la precisión final de la red.

<img src="imgs/keras-logo.png" width=30%>

## Keras
https://keras.io/

Keras es una API de redes neuronales de alto nivel, escrita en Python y capaz de ejecutarse sobre TensorFlow, CNTK o Theano. Fue desarrollado con la idea de permitir la experimentación rápida, poder pasar de la idea al resultado en el menor tiempo posible.

Veamos cómo montar una red en Keras para clasificar el dataset Iris. En primer lugar importamos la librería y el dataset. La función `LabelBinarizer` nos convertirá las etiquetas numéricas enteras a codificación one-hot.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

#load the iris dataset
iris = load_iris()

X = iris.data
encoder = LabelBinarizer()
y = encoder.fit_transform(iris.target)  # We transform to one-hot
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Creamos la red con la capa de entrada correspondiente a los cuatro valores de cada muestra. Una capa oculta de cinco neuronas y una capa de salida de tres neuronas. Aplicamos la función de activación **sigmoide** para la capa oculta y, dado que tenemos más de dos clases, **softmax** para la salida. Utilizamos *Stochastic Gradient Descent* como método optimizador con un **learning rate** de 0.01.

In [ ]:
from keras.models import Sequential #Sequential Models
from keras.layers import Dense #Dense Fully Connected Layer Type
from keras.optimizers import SGD #Stochastic Gradient Descent Optimizer

def create_network():
    model = Sequential()
    model.add(Dense(5, input_shape=(4,), activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))
        
    #stochastic gradient descent
    sgd = SGD(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

Entrenamos realizando 500 épocas sobre el conjunto de entrenamiento. Utilizamos un tamaño del lote (`batch_size`) de 10. Esto quiere decir que actualizamos todos los pesos de nuetra red utilizando solo 10 muestas.

In [ ]:
neural_network = create_network()
neural_network.fit(X_train, y_train, epochs=500, batch_size=10)

Echemos un vistazo a los resultados.

In [ ]:
print(neural_network.metrics_names)
neural_network.evaluate(X_test, y_test, verbose=1)

In [ ]:
predictions = neural_network.predict(X_test)
for p, l in zip(predictions, y_test):
    print(p,"->", l)

Una vez creado y entrenado nuestro modelo querremos guardarlo para usarlo en producción.

In [ ]:
from keras.models import load_model

neural_network.save("mimodelo.h5")

Supongamos que apagamos nuestro ordenador (`del neural_network`). Si queremos cargarlo de nuevo haremos:

In [ ]:
del neural_network  # deletes the existing model

# we return a compiled model
# identical to the previous one
model = load_model('mimodelo.h5')
print(model.metrics_names)
model.evaluate(X_test, y_test, verbose=1)

### Ejercicios
- Crea una red para clasificar el conjunto MNIST

# Redes Neuronales Recurrentes

In [ ]:
import sampledata

batch_size = 10
number_of_neurons = 5
digits = 8

data, label = sampledata.create_data(500)
print(data.shape)
print(label.shape)
label = label.reshape((500, 8, 1))
print(label.shape)

# Model ------------------------------------------------
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN, TimeDistributed
from keras import optimizers

model = Sequential()
model.add(SimpleRNN(number_of_neurons, input_shape=(data.shape[1:]), activation='sigmoid', return_sequences=True, use_bias=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))

sgd = optimizers.SGD(lr=1.)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

# train RNN
model.fit(data, label, epochs=100, batch_size=batch_size, verbose=1)

Veamos qué tal ha aprendido a sumar nuestra RNN.

In [ ]:
# Test ----------------------------------------------------

data, label = sampledata.create_data(100)
label = label.reshape((100, 8, 1))

error = model.evaluate(data, label, verbose=2)

data, label = sampledata.create_data(2)
predicted = np.round(model.predict(data))

for d, l, r in zip(data, label, predicted.reshape(2, 8)):
    print("data:")
    print(" ", np.transpose(d)[0][::-1])
    print("+", np.transpose(d)[1][::-1])
    print("------------------------------")
    print(" ", r[::-1], "result")
    print(" ", l[::-1], "label")
    print()
    print()

print("Precisión: "+ str(error[1]*100) + "%")
print("------------------------------")




## Ajuste a una curva

In [ ]:
from NN4Teaching import nn4t
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

x = np.arange(0.0, 10.0, 0.1)
secuencia = (np.sin(x-1)*0.5+0.1*np.sin(5*x)+1.5)*0.2 # Nuestra secuencia de valores


In [ ]:
plt.plot(x, secuencia)
plt.xlabel('x')
plt.ylabel('valores')
plt.title('Secuencia de valores')
plt.grid(True)
plt.show()

longitud_muestra = 10 #Elegido arbitrariamente

muestras_vector = []
etiquetas_vector = []
for i in range(len(secuencia)-longitud_muestra):
    muestras_vector.append(secuencia[i:i+longitud_muestra])
    etiquetas_vector.append([secuencia[i+longitud_muestra]])
    
    
    
print(muestras_vector[0])
print(etiquetas_vector[0])
print("----------------------")
print(muestras_vector[1])
print(etiquetas_vector[1])
print("----------------------")
print(muestras_vector[2])
print(etiquetas_vector[2])

In [ ]:
net = nn4t.Net(layers=[longitud_muestra, 5, 1])

for i in range(5000):
    net.train(muestras_vector, etiquetas_vector)
    
    if i%500 == 0:
        print("Epoch: ", i)
        print(net.loss(muestras_vector, etiquetas_vector))

In [ ]:
resultados = []

for i in range(longitud_muestra):
    resultados.append(muestras_vector[0][i])
        
for i in range(len(secuencia)-longitud_muestra):
    resultados.append(net.output(resultados[i:i+longitud_muestra])[0])
   
etiquetas_vector = ([[0]]*longitud_muestra) + etiquetas_vector

plt.plot(resultados, 'r-', etiquetas_vector, 'g-')
plt.xlabel('Longitud de muestra')
plt.ylabel('Valores')
plt.title('Resultados')
plt.grid(True)
plt.show()

## Ejercicios

- Realizar la predicción de las <a href="./data/solar_spots.csv">manchas solares</a>
- Medir el error medio y desviación típica